In [ ]:
import os
import time
import numpy as np

from dask import dataframe as dd
import dask_histogram

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LogNorm
from copy import copy

from iminuit import Minuit

In [ ]:
filepath = "/DATA_MASTER_PATH/2308_zoptical-zanalyze_final_vars/pred2_norm_conv1plus_withgencls_FINAL/"

def filesel(s):
    sel = True
    return sel

files_2pi = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("2p0_mal" in s))]
files_sig = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("sig_mal" in s))]
files_lam = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("lambda" in s))]

In [ ]:
def histsum(files):

    n_decays = 0

    for ifile, file, in enumerate(files):
            
        n_decays_0 = n_decays
        for i in range(1, 10):
            if "_%dM_"%(i) in file:
                n_decays += i*1e6
            if "_%dM_"%(10*i) in file:
                n_decays += i*10e6
            if "_%dM_"%(100*i) in file:
                n_decays += i*100e6
            if "_%dG_"%(i) in file:
                n_decays += i*1e9
            if "_%dG_"%(10*i) in file:
                n_decays += i*10e9
        print("opened %d/%d, with %d new decays" % (ifile+1, len(files), n_decays-n_decays_0))

        if ifile==0:
            #preds = pd.read_csv(filepath+file)
            preds = dd.read_csv(filepath+file)

        else:
            #preds = pd.concat((preds, pd.read_csv(filepath+file)))
            preds0 = dd.read_csv(filepath+file)
            preds = dd.multi.concat([preds, preds0])

    return preds, n_decays
    
preds_sig, n_decays_sig = histsum(files_sig)
preds_2pi, n_decays_2pi = histsum(files_2pi)
preds_lam, n_decays_lam = histsum(files_lam)

# working with this dataframe from now on
preds_tot = preds_sig.assign(cls=1)
preds_tot = dd.multi.concat([preds_tot, preds_2pi.assign(cls=0)])
preds_tot = dd.multi.concat([preds_tot, preds_lam.assign(cls=5)])
n_decays_tot = {
    1 : n_decays_sig,
    0 : n_decays_2pi,
    5 : n_decays_lam,
}

In [ ]:
preds_tot.describe()

In [ ]:
n_decays_tot

In [ ]:
# all general settings here

bnorm = False

clslabels = {
    1 : "k-pinunu",
    0 : "k-2pi",
    5 : "lambda-pin",
}

clscolors = {
    1 : "Blues",
    0 : "Oranges",
    5 : "Greens",
}

normfact = {
    1 : 15483/n_decays_tot[1], #37800/n_decays_tot[1], #15483/n_decays_tot[1],
    0 : 446e9/n_decays_tot[0], #1089e9/n_decays_tot[0], #446e9/n_decays_tot[0],
    5 : 8.22e13/n_decays_tot[5], #20e13/n_decays_tot[5], #8.22e13/n_decays_tot[5],
}

sig_sel = (0.503, 0.50715743073963)

In [ ]:
# add classifier predictions to main dataframe (pred0/1_bool)
preds_tot = preds_tot.assign(pred0_bool=(preds_tot["pred0"]>sig_sel[0]))
preds_tot = preds_tot.assign(pred1_bool=(preds_tot["pred1"]>sig_sel[1]))

In [ ]:
# add squared weights to main dataframe (Wsq)
preds_tot = preds_tot.assign(Wsq=(preds_tot["W"]*preds_tot["W"]))

---

In [ ]:
# settings of all other kinematical phase spaces
bins_ptth = (100, 200)
edges_ptth = ((0, 0.4), (0, 0.01)) 
box_edges_ptth = (0.14, 10, -1000, 1000)

bins_ept = (200, 100)
edges_ept = ((0, 200), (0, 0.4)) 
box_edges_ept = (-1000, 1000, 0.14, 10)

bins_rminpt = (200, 100)
edges_rminpt = ((0, 1.2), (0, 0.4)) 
box_edges_rminpt = (-1000, 1000, 0.14, 10)

In [ ]:
time0 = time.time()  # pt-th hist.

In [ ]:
# pt-th hist.
# no real-life statistics weight here

hists_ptth = {}

# true label: signal
preds = preds_tot[preds_tot["cls"]==1]
hists_ptth.update({1 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_pRecPi_T"], y=preds[boolsig]["Vertex_RecPi_sTh"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[1].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_pRecPi_T"], y=preds[~boolsig]["Vertex_RecPi_sTh"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[1].update({0 : np.array([x.compute() for x in hist])})

# true label: background (2pi - label 990)
preds = preds_tot[preds_tot["cls"]==0]
hists_ptth.update({990 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_pRecPi_T"], y=preds[boolsig]["Vertex_RecPi_sTh"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[990].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_pRecPi_T"], y=preds[~boolsig]["Vertex_RecPi_sTh"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[990].update({0 : np.array([x.compute() for x in hist])})

# true label: background (lambda - label 995)
preds = preds_tot[preds_tot["cls"]==5]
hists_ptth.update({995 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_pRecPi_T"], y=preds[boolsig]["Vertex_RecPi_sTh"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[995].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_pRecPi_T"], y=preds[~boolsig]["Vertex_RecPi_sTh"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ptth[0][0], edges_ptth[0][1], bins_ptth[0]), np.linspace(edges_ptth[1][0], edges_ptth[1][1], bins_ptth[1])),
)
hists_ptth[995].update({0 : np.array([x.compute() for x in hist])})

In [ ]:
# e-pt hist.
# no real-life statistics weight here

hists_ept = {}

# true label: signal
preds = preds_tot[preds_tot["cls"]==1]
hists_ept.update({1 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_ERecPi"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[1].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_ERecPi"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[1].update({0 : np.array([x.compute() for x in hist])})

# true label: background (2pi - label 990)
preds = preds_tot[preds_tot["cls"]==0]
hists_ept.update({990 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_ERecPi"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[990].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_ERecPi"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[990].update({0 : np.array([x.compute() for x in hist])})

# true label: background (lambda - label 995)
preds = preds_tot[preds_tot["cls"]==5]
hists_ept.update({995 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Vertex_ERecPi"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[995].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Vertex_ERecPi"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_ept[0][0], edges_ept[0][1], bins_ept[0]), np.linspace(edges_ept[1][0], edges_ept[1][1], bins_ept[1])),
)
hists_ept[995].update({0 : np.array([x.compute() for x in hist])})

In [ ]:
# rmin-pt hist.
# no real-life statistics weight here

hists_rminpt = {}

# true label: signal
preds = preds_tot[preds_tot["cls"]==1]
hists_rminpt.update({1 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Clusters_xRec_TMin"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[1].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Clusters_xRec_TMin"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[1].update({0 : np.array([x.compute() for x in hist])})

# true label: background (2pi - label 990)
preds = preds_tot[preds_tot["cls"]==0]
hists_rminpt.update({990 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Clusters_xRec_TMin"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[990].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Clusters_xRec_TMin"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[990].update({0 : np.array([x.compute() for x in hist])})

# true label: background (lambda - label 995)
preds = preds_tot[preds_tot["cls"]==5]
hists_rminpt.update({995 : {}})
boolsig = (preds["pred0_bool"]) & (preds["pred1_bool"])  

# --> pred. label: signal
hist = dask_histogram.histogram2d(
    x=preds[boolsig]["Clusters_xRec_TMin"], y=preds[boolsig]["Vertex_pRecPi_T"], weights=preds[boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[995].update({1 : np.array([x.compute() for x in hist])})

# --> pred. label: background (any)
hist = dask_histogram.histogram2d(
    x=preds[~boolsig]["Clusters_xRec_TMin"], y=preds[~boolsig]["Vertex_pRecPi_T"], weights=preds[~boolsig]["W"],
    bins=(np.linspace(edges_rminpt[0][0], edges_rminpt[0][1], bins_rminpt[0]), np.linspace(edges_rminpt[1][0], edges_rminpt[1][1], bins_rminpt[1])),
)
hists_rminpt[995].update({0 : np.array([x.compute() for x in hist])})

In [ ]:
time1 = time.time()  # pt-th hist.
print("execution time: %f" % (time1-time0))

In [ ]:
# all other kinematical phase spaces again
# real-life statistics weight is applied here, and stays below

# true label: background (any - label 0)
hists_ptth[0] = {
    1 : [
        hists_ptth[990][1][0]*normfact[0] + hists_ptth[995][1][0]*normfact[5],
        hists_ptth[990][1][1],
        hists_ptth[990][1][2]
    ],
    0 : [
        hists_ptth[990][0][0]*normfact[0] + hists_ptth[995][0][0]*normfact[5],
        hists_ptth[990][0][1],
        hists_ptth[990][0][2]
    ],
}

hists_ept[0] = {
    1 : [
        hists_ept[990][1][0]*normfact[0] + hists_ept[995][1][0]*normfact[5],
        hists_ept[990][1][1],
        hists_ept[990][1][2]
    ],
    0 : [
        hists_ept[990][0][0]*normfact[0] + hists_ept[995][0][0]*normfact[5],
        hists_ept[990][0][1],
        hists_ept[990][0][2]
    ],
}

hists_rminpt[0] = {
    1 : [
        hists_rminpt[990][1][0]*normfact[0] + hists_rminpt[995][1][0]*normfact[5],
        hists_rminpt[990][1][1],
        hists_rminpt[990][1][2]
    ],
    0 : [
        hists_rminpt[990][0][0]*normfact[0] + hists_rminpt[995][0][0]*normfact[5],
        hists_rminpt[990][0][1],
        hists_rminpt[990][0][2]
    ],
}

# also, rescale true signal
hists_ptth[1][1][0] *= normfact[1]
hists_ptth[1][0][0] *= normfact[1]

hists_ept[1][1][0] *= normfact[1]
hists_ept[1][0][0] *= normfact[1]

hists_rminpt[1][1][0] *= normfact[1]
hists_rminpt[1][0][0] *= normfact[1]

In [ ]:
# another expression of the plot edges (obtained from the histograms)
extent_ptth = [
    min(hists_ptth[1][1][1]), max(hists_ptth[1][1][1]), min(hists_ptth[1][1][2]), max(hists_ptth[1][1][2])
]

extent_ept = [
    min(hists_ept[1][1][1]), max(hists_ept[1][1][1]), min(hists_ept[1][1][2]), max(hists_ept[1][1][2])
]

extent_rminpt = [
    min(hists_rminpt[1][1][1]), max(hists_rminpt[1][1][1]), min(hists_rminpt[1][1][2]), max(hists_rminpt[1][1][2])
]

# fiducial volume box to be superimposed on the plots
box_ptth = patches.Rectangle(
    (box_edges_ptth[0], box_edges_ptth[2]), box_edges_ptth[1]-box_edges_ptth[0], box_edges_ptth[3]-box_edges_ptth[2],
    linewidth=1, edgecolor='0.5', facecolor='none', label="LoI fiducial box"
)

box_ept = patches.Rectangle(
    (box_edges_ept[0], box_edges_ept[2]), box_edges_ept[1]-box_edges_ept[0], box_edges_ept[3]-box_edges_ept[2],
    linewidth=1, edgecolor='0.5', facecolor='none', label="LoI fiducial box"
)

box_rminpt = patches.Rectangle(
    (box_edges_rminpt[0], box_edges_rminpt[2]), box_edges_rminpt[1]-box_edges_rminpt[0], box_edges_rminpt[3]-box_edges_rminpt[2],
    linewidth=1, edgecolor='0.5', facecolor='none', label="LoI fiducial box"
)

In [ ]:
classnames = {
    995 : "lambda-pin",
    990 : "k-2pi",
    1 : "k-pinunu",
    0 : "background",
}

cmap = plt.get_cmap("jet").copy()
cmap.set_bad('white')

In [ ]:
# pt-th plots - one per true label
# real-life statistics weight is applied here, after rewriting hists_ptth

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

ax = axs[0]
ax.set_title("true background")
ax.set_xlim((extent_ptth[0], extent_ptth[1]))
ax.set_ylim((extent_ptth[2], extent_ptth[3]))
ax.add_patch(copy(box_ptth))

data = hists_ptth[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="pred. as background")

data = hists_ptth[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="pred. as signal")

ax = axs[1]
ax.set_title("true signal")
ax.set_xlim((extent_ptth[0], extent_ptth[1]))
ax.set_ylim((extent_ptth[2], extent_ptth[3]))
ax.add_patch(copy(box_ptth))

data = hists_ptth[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_ptth[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# pt-th plots - one per predicted label
# real-life statistics weight is applied here, after rewriting hists_ptth

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

pred_class = 0
ax = axs[pred_class]
ax.set_title("predicted as background")
ax.set_xlim((extent_ptth[0], extent_ptth[1]))
ax.set_ylim((extent_ptth[2], extent_ptth[3]))
ax.add_patch(copy(box_ptth))

data = hists_ptth[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="true background")

data = hists_ptth[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="true signal")

pred_class = 1
ax = axs[pred_class]
ax.set_title("predicted as signal")
ax.set_xlim((extent_ptth[0], extent_ptth[1]))
ax.set_ylim((extent_ptth[2], extent_ptth[3]))
ax.add_patch(copy(box_ptth))

data = hists_ptth[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_ptth[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# pt-th plots - individual classes separately, all true lables and only signal-selected
# real-life statistics weight is applied here, after rewriting hists_ptth

for iclass in (1, 990, 995):
    
    fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
    blog = False

    ax = axs[0]
    ax.set_title("all")
    ax.set_xlim((extent_ptth[0], extent_ptth[1]))
    ax.set_ylim((extent_ptth[2], extent_ptth[3]))
    ax.add_patch(copy(box_ptth))

    data = hists_ptth[iclass][1][0].T + hists_ptth[iclass][0][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(121)
    plt.colorbar(cax)

    ax = axs[1]
    ax.set_title("predicted as %s" % classnames[1])
    ax.set_xlim((extent_ptth[0], extent_ptth[1]))
    ax.set_ylim((extent_ptth[2], extent_ptth[3]))
    ax.add_patch(copy(box_ptth))

    data = hists_ptth[iclass][1][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_ptth, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(122)
    plt.colorbar(cax)

    fig.suptitle(classnames[iclass])
    fig.tight_layout()

In [ ]:
# e-pt plots - one per true label
# real-life statistics weight is applied here, after rewriting hists_ept

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

ax = axs[0]
ax.set_title("true background")
ax.set_xlim((extent_ept[0], extent_ept[1]))
ax.set_ylim((extent_ept[2], extent_ept[3]))
ax.add_patch(copy(box_ept))

data = hists_ept[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="pred. as background")

data = hists_ept[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="pred. as signal")

ax = axs[1]
ax.set_title("true signal")
ax.set_xlim((extent_ept[0], extent_ept[1]))
ax.set_ylim((extent_ept[2], extent_ept[3]))
ax.add_patch(copy(box_ept))

data = hists_ept[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_ept[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# e-pt plots - one per predicted label
# real-life statistics weight is applied here, after rewriting hists_ept

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

pred_class = 0
ax = axs[pred_class]
ax.set_title("predicted as background")
ax.set_xlim((extent_ept[0], extent_ept[1]))
ax.set_ylim((extent_ept[2], extent_ept[3]))
ax.add_patch(copy(box_ept))

data = hists_ept[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="true background")

data = hists_ept[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="true signal")

pred_class = 1
ax = axs[pred_class]
ax.set_title("predicted as signal")
ax.set_xlim((extent_ept[0], extent_ept[1]))
ax.set_ylim((extent_ept[2], extent_ept[3]))
ax.add_patch(copy(box_ept))

data = hists_ept[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_ept[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# e-pt plots - individual classes separately, all true lables and only signal-selected
# real-life statistics weight is applied here, after rewriting hists_ept

for iclass in (1, 990, 995):
    
    fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
    blog = False

    ax = axs[0]
    ax.set_title("all")
    ax.set_xlim((extent_ept[0], extent_ept[1]))
    ax.set_ylim((extent_ept[2], extent_ept[3]))
    ax.add_patch(copy(box_ept))

    data = hists_ept[iclass][1][0].T + hists_ept[iclass][0][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(121)
    plt.colorbar(cax)

    ax = axs[1]
    ax.set_title("predicted as %s" % classnames[1])
    ax.set_xlim((extent_ept[0], extent_ept[1]))
    ax.set_ylim((extent_ept[2], extent_ept[3]))
    ax.add_patch(copy(box_ept))

    data = hists_ept[iclass][1][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_ept, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(122)
    plt.colorbar(cax)

    fig.suptitle(classnames[iclass])
    fig.tight_layout()

In [ ]:
# rmin-pt plots - one per true label
# real-life statistics weight is applied here, after rewriting hists_rminpt

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

ax = axs[0]
ax.set_title("true background")
ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
ax.add_patch(copy(box_rminpt))

data = hists_rminpt[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="pred. as background")

data = hists_rminpt[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="pred. as signal")

ax = axs[1]
ax.set_title("true signal")
ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
ax.add_patch(copy(box_rminpt))

data = hists_rminpt[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_rminpt[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# rmin-pt plots - one per predicted label
# real-life statistics weight is applied here, after rewriting hists_rminpt

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
blog = True

pred_class = 0
ax = axs[pred_class]
ax.set_title("predicted as background")
ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
ax.add_patch(copy(box_rminpt))

data = hists_rminpt[0][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="true background")

data = hists_rminpt[1][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="true signal")

pred_class = 1
ax = axs[pred_class]
ax.set_title("predicted as signal")
ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
ax.add_patch(copy(box_rminpt))

data = hists_rminpt[0][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_rminpt[1][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
# rmin-pt plots - individual classes separately, all true lables and only signal-selected
# real-life statistics weight is applied here, after rewriting hists_rminpt

for iclass in (1, 990, 995):
    
    fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
    blog = False

    ax = axs[0]
    ax.set_title("all")
    ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
    ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
    ax.add_patch(copy(box_rminpt))

    data = hists_rminpt[iclass][1][0].T + hists_rminpt[iclass][0][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(121)
    plt.colorbar(cax)

    ax = axs[1]
    ax.set_title("predicted as %s" % classnames[1])
    ax.set_xlim((extent_rminpt[0], extent_rminpt[1]))
    ax.set_ylim((extent_rminpt[2], extent_rminpt[3]))
    ax.add_patch(copy(box_rminpt))

    data = hists_rminpt[iclass][1][0].T
    hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
    cax = ax.matshow(hist, origin="lower", extent=extent_rminpt, aspect="auto", norm=LogNorm() if blog else None, cmap=cmap)
    plt.subplot(122)
    plt.colorbar(cax)

    fig.suptitle(classnames[iclass])
    fig.tight_layout()